<a href="https://colab.research.google.com/github/ejlee127/sound-learning/blob/main/train_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (146 kB/s)
Reading package lists... Done


In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-11-11 20:23:41--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2020-11-11 20:23:42 (8.62 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



UrbanSound Machine Learning: Experiment 4


*   Inputs: MFCCs statistical values


> input tables : fold1_mfcc_stat_n_180.csv  ~  fold10_mfcc_stat_n_180.csv

> mfcc : MFCC feature

> stat : scipy.stats.describe values -- min, max, mean, var, skewedness, kurtosis

> n_180 : each sample is an array of length 180

*   Model : SVM - rbf


> https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html




In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [6]:
import numpy as np
import pandas as pd
import json

#import tensorflow.keras as keras
#from keras.models import Sequential
#from keras.layers import Dense, Dropout, Activation
#from keras.optimizers import Adam
#from keras.utils import to_categorical

from sklearn.svm import SVC
from sklearn.metrics import classification_report
# cross validation
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

In [7]:
# Mount google drive. Hit this cell and then click the link. Follow the instruction.
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Set the path

# project top folder
myproject_path = "/content/drive/My Drive/BootCamp/Project3-UrbanSound/"

# dataset folder: under this folder, there are 10 files named fold{i}_mfcc_stat_n_180.csv
dataset_path = os.path.join(myproject_path, "datasets/sql_tables/mfcc_stat_tables/")

# folder to store the output
output_path = os.path.join(myproject_path, "exp4_results/")


In [12]:
# Define functions

# create_train_test_val_set():
#   returns train set of [X, y], test set of [X, y] and val set of [X, y]
#   tables : list of mfcc_avg_n_40 tables
#   test_index : 
#   val_index :
def create_train_test_set(tables, train_index, test_index):

  #train_list = [ i for i in range(1,NUMFOLDS+1) if i not in TESTFOLD ]
  train_df = pd.DataFrame()
  test_df = pd.DataFrame()
  #val_df = pd.DataFrame()
  print("train folders:", train_index)
  print("test folders:", test_index)

  for i in range(1,NUMFOLDS+1):
    # load the table of fold{i}
    df_pandas = tables[i-1].toPandas()

    if i in train_index:     
      train_df = pd.concat( [train_df, df_pandas] )
    else:
      test_df = pd.concat( [test_df, df_pandas] )

  X_train = train_df.drop(['_c0', 'slice_fn', 'classID'], axis = 1)
  y_train = train_df['classID']
  train_Xy = [X_train.astype('float32'), y_train.astype('int')]

  X_test = test_df.drop(['_c0', 'slice_fn', 'classID'], axis = 1)
  y_test = test_df['classID']
  test_Xy = [X_test.astype('float32'), y_test.astype('int')]

  #X_val = val_df.drop(['_c0', 'fileID', 'classID'], axis = 1)
  #y_val = val_df['classID']
  #val_Xy = [X_val.astype('float32'), y_val.astype('int')]

  print("X_train shape: ", X_train.shape)
  print("y_train shape: ", y_train.shape)
  print("X_test shape: ", X_test.shape)
  print("y_test shape: ", y_test.shape)
  return train_Xy, test_Xy


#-----------------------------
# Define the parameters for the model training and classification report
class_names=["air_conditioner",
             "car_horn",
             "children_playing",
             "dog_bark",
             "drilling",
             "engine_idling",
             "gun_shot",
             "jackhammer",
             "siren",
             "street_music"]

numClasses = 10 # number of target neurons
#-----------------------------

# train_Model_save_Result:
#    run SVC(rbf) model for the set of parameters, and save the result in a file
def train_Model_save_Result(X_train, y_train, X_test, y_test, scaled=True):


  #print("X_train shape=", X_train.shape, X_train.dtypes)
  #print("y_train shape=", y_train.shape, y_train.dtypes)
  #print("X_test shape=", X_test.shape, X_test.dtypes)
  #print("y_test shape=", y_test.shape, y_test)

  if scaled:
    # Transform using standard scaler
    print("inputs are scaled.")
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_test = ss.fit_transform(X_test)
  
  svc_model = SVC(kernel='rbf')
  svc_model.fit(X_train, y_train)
  predictions = svc_model.predict(X_test)

  # Predict with X-test
  score = svc_model.score(X_test, y_test)
  print("score=", score)
  print(classification_report(y_test, predictions,
                            target_names=class_names))

  report = classification_report(y_test, predictions,
                                target_names=class_names,
                                output_dict=True)

  return score, report

In [10]:
NUMFOLDS = 10
# Read mfcc_avg_n_40 tables from csv files
list_tables = []
for i in range(1,NUMFOLDS+1):
  fn = f"fold{i}_mfcc_stat_n_180.csv"
  filepath = os.path.join(dataset_path, fn)
  table = spark.read.option("header", "true").csv(filepath)
  list_tables.append(table)

# check whether the tables are read
list_tables[3].show(3)


+---+-----------------+-------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+--------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+-----------

In [13]:

# Set the folders to train and test using KFold for the cross-validation
rn = np.array(range(1,NUMFOLDS+1))
kf10 = KFold(n_splits=10, shuffle=False)

# Train model and save the results
svc_results = []
scores = []

# The inputs are transformed using StandarScaler. If not, take the commented out line.
resultfile = os.path.join(output_path, f"exp4_report_scaled.json")
#resultfile = os.path.join(output_path, f"exp4_report.json")

for train_idx, test_idx in kf10.split(rn):
  #  folder number = idx + 1 (since folder is from 1 to 10)
  train_Xy, test_Xy= create_train_test_set(list_tables, train_idx+1, test_idx+1)
  score, report = train_Model_save_Result(train_Xy[0], train_Xy[1], test_Xy[0], test_Xy[1], scaled=True)
  svc_results.append( report )
  scores.append(score)
  #print(svc_results)
  print("folder and score:", test_idx+1, score)

# Save the result  
with open(resultfile, "w") as f:
  json.dump(svc_results, f)

train folders: [ 2  3  4  5  6  7  8  9 10]
test folders: [1]
X_train shape:  (7859, 180)
y_train shape:  (7859,)
X_test shape:  (873, 180)
y_test shape:  (873,)
inputs are scaled.
score= 0.6025200458190149
                  precision    recall  f1-score   support

 air_conditioner       0.49      0.42      0.45       100
        car_horn       0.80      0.97      0.88        36
children_playing       0.68      0.80      0.73       100
        dog_bark       0.62      0.75      0.68       100
        drilling       0.45      0.57      0.50       100
   engine_idling       0.60      0.46      0.52        96
        gun_shot       0.84      0.74      0.79        35
      jackhammer       0.33      0.26      0.29       120
           siren       0.87      0.76      0.81        86
    street_music       0.69      0.71      0.70       100

        accuracy                           0.60       873
       macro avg       0.64      0.64      0.63       873
    weighted avg       0.60      0.60